## Taste of wine

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
seed = 2021
np.random.seed(seed)
tf.random.set_seed(seed)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
cd /content/drive/MyDrive/colab

/content/drive/MyDrive/colab


In [ ]:
wine = pd.read_csv('dataset/wine.csv', header = None)
wine

,0,1,2,3,4,5,6,7,8,9,10,11,12
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.99780,3.51,0.56,9.4,5,1
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.99680,3.20,0.68,9.8,5,1
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.99700,3.26,0.65,9.8,5,1
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.99800,3.16,0.58,9.8,6,1
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.99780,3.51,0.56,9.4,5,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
6492,6.2,0.21,0.29,1.6,0.039,24.0,92.0,0.99114,3.27,0.50,11.2,6,0
6493,6.6,0.32,0.36,8.0,0.047,57.0,168.0,0.99490,3.15,0.46,9.6,5,0
6494,6.5,0.24,0.19,1.2,0.041,30.0,111.0,0.99254,2.99,0.46,9.4,6,0
6495,5.5,0.29,0.30,1.1,0.022,20.0,110.0,0.98869,3.34,0.38,12.8,7,0


### Preprocessing

In [ ]:
wine = wine[wine[11] != 3]
wine = wine[wine[11] != 9]

In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(wine.iloc[:,: -2].values)

In [ ]:
from tensorflow.keras.utils import to_categorical

y = to_categorical(wine.iloc[:, -2].values)
y

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 1., 0., 0.]], dtype=float32)

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, Y_train, Y_test = train_test_split(
    X_scaled, y, stratify = y, random_state = seed
)

X_train.shape, X_test.shape, Y_train.shape, Y_test.shape

((4846, 11), (1616, 11), (4846, 9), (1616, 9))

### Processing

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [ ]:
model = Sequential()
model.add(Dense(30, input_dim = 11, activation = 'relu'))
model.add(Dense(20, activation = 'relu'))
model.add(Dense(15, activation = 'relu'))
model.add(Dense(9, activation = 'softmax'))
model.summary()

Model: "sequential_15"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_60 (Dense)             (None, 30)                360       
_________________________________________________________________
dense_61 (Dense)             (None, 20)                620       
_________________________________________________________________
dense_62 (Dense)             (None, 15)                315       
_________________________________________________________________
dense_63 (Dense)             (None, 9)                 144       
Total params: 1,439
Trainable params: 1,439
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.compile(
    optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy']
)

#### Setting for saving model

In [ ]:
modelpath = '1. Basic Model/Models/best_taste.h5'

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint

checkpointer = ModelCheckpoint(
    modelpath, monitor = 'val_loss', verbose = 1, save_best_only = True
)

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping
early_stopping = EarlyStopping(patience = 50)

#### Learing and saving model

In [ ]:
history = model.fit(
    X_train, Y_train, validation_split = 0.2,
    epochs = 1000, batch_size = 200, verbose = 0, callbacks = [checkpointer, early_stopping]
)


Epoch 00001: val_loss improved from inf to 2.14117, saving model to 1. Basic Model/Models/best_taste.h5

Epoch 00002: val_loss improved from 2.14117 to 2.04210, saving model to 1. Basic Model/Models/best_taste.h5

Epoch 00003: val_loss improved from 2.04210 to 1.86803, saving model to 1. Basic Model/Models/best_taste.h5

Epoch 00004: val_loss improved from 1.86803 to 1.66731, saving model to 1. Basic Model/Models/best_taste.h5

Epoch 00005: val_loss improved from 1.66731 to 1.47241, saving model to 1. Basic Model/Models/best_taste.h5

Epoch 00006: val_loss improved from 1.47241 to 1.32135, saving model to 1. Basic Model/Models/best_taste.h5

Epoch 00007: val_loss improved from 1.32135 to 1.25583, saving model to 1. Basic Model/Models/best_taste.h5

Epoch 00008: val_loss improved from 1.25583 to 1.21456, saving model to 1. Basic Model/Models/best_taste.h5

Epoch 00009: val_loss improved from 1.21456 to 1.18148, saving model to 1. Basic Model/Models/best_taste.h5

Epoch 00010: val_loss 

#### Evaluating with the best model

In [ ]:
from tensorflow.keras.models import load_model
best_model = load_model('1. Basic Model/Models/best_taste.h5')

In [ ]:
best_model.evaluate(X_test, Y_test)

51/51 [==============================] - 0s 1ms/step - loss: 1.0242 - accuracy: 0.5501


[1.0242393016815186, 0.5501237511634827]